<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/02_sintaxis_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [2]:
#! pip install polars -U

# Lazy & Collect

In [3]:
import polars as pl

Creemos un dataframe que se conporte a nivel `sintaxis` como `lazy` aunque realmente las operaciones de creacion ya han sido ejecutadas.  
En estricto sentido peudes leer los datos como `lazy` desde el disco, pero para entender como funciona haremos un mock.

In [4]:
# Crear un DataFrame con evaluación perezosa
lazy_df = pl.DataFrame({
    "nombre": ["Ana", "Luis", "Marta"],
    "edad": [25, 32, 18],
    "ciudad": ["Madrid", "Barcelona", "Sevilla"]
}).lazy()


In [7]:
# Seleccionar solo la columna 'nombre'
seleccion = lazy_df.select("nombre")


In [8]:
print(lazy_df)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

DF ["nombre", "edad", "ciudad"]; PROJECT */3 COLUMNS; SELECTION: "None"


Que esta haciendo el `select`?

In [12]:
seleccion=lazy_df.select('edad')
seleccion

<LazyFrame [1 col, {"edad": Int64}] at 0x121ACF8D0>

`collect` ejectuta todas las operaciones lazy que no han sido ejecutadas hasta es momento. Digamos que es un comando que indica a `polars` que es momento de ejecutar.

In [16]:
lazy_df.explain(optimized=True)

'DF ["nombre", "edad", "ciudad"]; PROJECT */3 COLUMNS; SELECTION: "None"'

In [17]:
# Ejecutar y mostrar los resultados
print(seleccion.collect())

shape: (3, 1)
┌──────┐
│ edad │
│ ---  │
│ i64  │
╞══════╡
│ 25   │
│ 32   │
│ 18   │
└──────┘


# Filtrado y Operaciones Columna

In [21]:
# Filtrar el DataFrame donde la edad es mayor a 20
filtrado = lazy_df.filter(pl.col("edad") > 20)

Que esta haciendo `filter`?  
Que esta haciendo `pl.col`?

In [22]:
# Crear una nueva columna que sea la longitud del nombre
nueva_columna = filtrado.with_columns(pl.col("nombre").str.lengths().alias("longitud_nombre"))

/var/folders/_m/zrywqjcn66j88_lz9f9pm4gc0000gn/T/ipykernel_51465/4089663324.py:2: DeprecationWarning: `lengths` is deprecated. It has been renamed to `len_bytes`.
  nueva_columna = filtrado.with_columns(pl.col("nombre").str.lengths().alias("longitud_nombre"))


Que esta haciendo `with_columns`?  
Por que aparece un `pl.col`?

In [26]:
nueva_columna.show_graph

<bound method LazyFrame.show_graph of <LazyFrame [4 cols, {"nombre": String … "longitud_nombre": UInt32}] at 0x1220E3A50>>

In [23]:
# Ejecutar y mostrar los resultados
print(nueva_columna.collect())

shape: (2, 4)
┌────────┬──────┬───────────┬─────────────────┐
│ nombre ┆ edad ┆ ciudad    ┆ longitud_nombre │
│ ---    ┆ ---  ┆ ---       ┆ ---             │
│ str    ┆ i64  ┆ str       ┆ u32             │
╞════════╪══════╪═══════════╪═════════════════╡
│ Ana    ┆ 25   ┆ Madrid    ┆ 3               │
│ Luis   ┆ 32   ┆ Barcelona ┆ 4               │
└────────┴──────┴───────────┴─────────────────┘


$$

$$

# Operaciones de agregacion

In [37]:
# Agrupar por 'ciudad' y calcular la edad promedio
agregacion = lazy_df.group_by("ciudad").agg([
    pl.col("edad").mean().alias("edad_promedio")
])


Que esta haciendo el `agg` en el `groupby`?  
Que esta haciendo el `alias`?

In [38]:
# Ejecutar y mostrar los resultados
print(agregacion.collect())

shape: (3, 2)
┌───────────┬───────────────┐
│ ciudad    ┆ edad_promedio │
│ ---       ┆ ---           │
│ str       ┆ f64           │
╞═══════════╪═══════════════╡
│ Sevilla   ┆ 18.0          │
│ Barcelona ┆ 32.0          │
│ Madrid    ┆ 25.0          │
└───────────┴───────────────┘


# Combinar Operaciones

In [39]:
# Combinar filtrado, agregación y selección de columnas
combinado = lazy_df.filter(pl.col("edad") > 20).groupby("ciudad").agg([
    pl.col("edad").mean().alias("edad_promedio")
]).select(["ciudad", "edad_promedio"])


/var/folders/_m/zrywqjcn66j88_lz9f9pm4gc0000gn/T/ipykernel_51465/945101761.py:2: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  combinado = lazy_df.filter(pl.col("edad") > 20).groupby("ciudad").agg([


In [40]:
# Ejecutar y mostrar los resultados
print(combinado.collect())


shape: (2, 2)
┌───────────┬───────────────┐
│ ciudad    ┆ edad_promedio │
│ ---       ┆ ---           │
│ str       ┆ f64           │
╞═══════════╪═══════════════╡
│ Madrid    ┆ 25.0          │
│ Barcelona ┆ 32.0          │
└───────────┴───────────────┘


In [41]:
lazy_df.group_by('ciudad').agg(pl.col('edad'), pl.col('nombre')).collect()

ciudad,edad,nombre
str,list[i64],list[str]
"""Sevilla""",[18],"[""Marta""]"
"""Madrid""",[25],"[""Ana""]"
"""Barcelona""",[32],"[""Luis""]"
